In [ ]:
%matplotlib inline

import math
import datetime

import numpy as np

from pandas import DataFrame
from pandas import concat

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['font.family'] = 'serif'

debug = True

In [ ]:
import pip 

cuda = False

l = next(str(i) for i in pip.get_installed_distributions() if 'tensorflow' in str(i))
if l.startswith('tensorflow-gpu'):
    cuda = True

In [ ]:
#MATLAB colors
colors = [[0, 0.4470, 0.7410],
          [0.8500, 0.3250, 0.0980],
          [0.9290, 0.6940, 0.1250],
          [0.4940, 0.1840, 0.5560],
          [0.4660, 0.6740, 0.1880],
          [0.3010, 0.7450, 0.9330],
          [0.6350, 0.0780, 0.1840]];

colormap = plt.get_cmap('rainbow')

def getColor(num, brightness=1):
    col = colors[num]
    return [col[0]*brightness, col[1]*brightness, col[2]*brightness]

In [ ]:
N = 25 #Number of experiments
size_of_msg = 0 #number of relevant bytes in a message (to be computed as we load the logs)
size_of_interaction = 3 #number of messages in complete interactions between client and server

def logToPositions(filename):
    global size_of_msg
    file = open(filename)
    comments = 0
    found_christmas = False
    while True:
        line = file.readline()
        if not line:
            break
        elif line.startswith('#CLI: What are you waitin for? Christmas?'):
            found_christmas = True
        elif line.startswith('#CLI: Heh, heh, heh... what a mess!'):
            raise Exception('There was an error in the execution of this protocol ' + filename)
        elif line.startswith('#CLI:') or line.startswith('#APP:'):
            continue
        elif line.startswith('#'):
            positions = line[1:].strip().split(',')[:-1]
            if (size_of_msg == 0):
                size_of_msg=len(positions)
            for p in positions:
                yield float(p)
    if not found_christmas:
        print('WARNING! Logs may not be complete for ' + filename)

In [ ]:
def  loadAllPositions(base, N, pad=False):
    all_positions = []
    max_len = 0
    for i in range(1,N+1):
        positions = list(logToPositions(base+str(i)+'.log'))
        if len(positions) > max_len:
            max_len = len(positions)
        all_positions.append(np.array(list(positions)))
    if pad:
        for i in range(0,len(all_positions)):
            all_positions[i] = np.pad(all_positions[i], (0, max_len-len(all_positions[i])), 'constant', constant_values=np.nan)        
    return all_positions

In [ ]:
base_dir = '../../../target/thingml-bytes-logs/nodejs/'

if debug:
    base_positions = np.array(loadAllPositions(base_dir+'base/nodejs', N, pad=True))
    static_positions = np.array(loadAllPositions(base_dir+'static/nodejs', N, pad=True))
    runtime_positions = np.array(loadAllPositions(base_dir+'dynamic/nodejs', N, pad=True))

    plt.figure(1, figsize=(15,6))
    plt.imshow(base_positions, interpolation='nearest', origin='lower', aspect='auto', cmap=colormap)
    plt.savefig('picture_raw_positions_base.png', dpi=300)
    plt.show()
    plt.figure(2, figsize=(15,6))
    plt.imshow(static_positions, interpolation='nearest', origin='lower', aspect='auto', cmap=colormap)
    plt.savefig('picture_raw_positions_static.png', dpi=300)
    plt.show()
    plt.figure(3, figsize=(15,6))
    plt.imshow(runtime_positions, interpolation='nearest', origin='lower', aspect='auto', cmap=colormap)
    plt.savefig('picture_raw_positions_runtime.png', dpi=300)
    plt.show()

In [ ]:
def split_train_test(data, train_ratio = 0.67):
    train_test = np.empty([len(data),2], dtype=object)
    for i in range(0,len(data)):
        train_size = int(len(data[i]) * train_ratio)
        test_size = len(data[i]) - train_size
        train, test = data[i][0:train_size], data[i][train_size:len(data[i])]
        train_test[i][0] = train 
        train_test[i][1] = test
    return train_test

#Scale data to fit into y=[0,1]
#train = MinMaxScaler().fit_transform(train.reshape(-1, 1))
#test = MinMaxScaler().fit_transform(test.reshape(-1, 1))

base_positions = np.array(loadAllPositions(base_dir+'base/nodejs', N))
static_positions = np.array(loadAllPositions(base_dir+'static/nodejs', N))
runtime_positions = np.array(loadAllPositions(base_dir+'dynamic/nodejs', N))

base = split_train_test(base_positions)
static = split_train_test(static_positions)
runtime = split_train_test(runtime_positions)

base_train = np.array(base[:, 0])
base_test = np.array(base[:, 1])
static_train = np.array(static[:, 0])
static_test = np.array(static[:, 1])
runtime_train = np.array(runtime[:, 0])
runtime_test = np.array(runtime[:, 1])

In [ ]:
number_of_interactions = 3
size_of_window = number_of_interactions * size_of_interaction * size_of_msg

"""
See https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
"""
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg   

In [ ]:
def reshape(train, test):
    train_X = np.array(train.iloc[:,0:-1])
    train_Y = np.array(train.iloc[:,-1])
    test_X = np.array(test.iloc[:,0:-1])
    test_Y = np.array(test.iloc[:,-1])

    train_Y = np.reshape(train_Y, (train_Y.shape[0],1))
    test_Y = np.reshape(test_Y, (test_Y.shape[0],1))

    #FIXME: make a function
    n = train_X.shape[0]
    L = 3
    X_train_seq = []
    Y_train_seq = []
    for k in range(n - L + 1):
        X_train_seq.append(train_X[k : k + L])
        Y_train_seq.append(train_Y[k : k + L])
    train_X = np.array(X_train_seq)
    train_Y = np.array(Y_train_seq)

    #FIXME: make a function
    n = test_X.shape[0]
    L = 3
    X_test_seq = []
    Y_test_seq = []
    for k in range(n - L + 1):
        X_test_seq.append(test_X[k : k + L])
        Y_test_seq.append(test_Y[k : k + L])
    test_X = np.array(X_test_seq)
    test_Y = np.array(Y_test_seq)
    
    return train_X, train_Y, test_X, test_Y

def keras_model(x, y):    
    model = keras.Sequential()
    model.add(keras.layers.Dense(x, input_shape=(y, x)))
    if cuda:
        model.add(keras.layers.Bidirectional(keras.layers.CuDNNLSTM(x, return_sequences=True)))
    else:
        model.add(keras.layers.Bidirectional(keras.layers.LSTM(x, return_sequences=True), input_shape=(y, x)))
    model.add(keras.layers.Dense(x))
    model.add(keras.layers.Dense(1))
    model.compile(loss="mse", optimizer="adam")
#     model.summary()
    return model

def train_and_save(model, train_X, train_Y, test_X, test_Y, mode, xp, save=False):
    earlystop_loss = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0025, patience=8, verbose=1)
    earlystop_val_loss = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0025, patience=8, verbose=1)
    callbacks_list = [earlystop_loss, earlystop_val_loss]
    
    model.fit(train_X, train_Y, batch_size=12, epochs=100, validation_split=0.1, verbose=1, callbacks=callbacks_list)

    if save:
        keras.models.save_model(model, base_dir + mode + str(xp) + '.h5', overwrite=True, include_optimizer=True)
#     if debug:
#         result = model.evaluate(test_X, test_Y)
#         print(result)
    return model

In [ ]:
def predict(model, test_X, test_Y):

    predictions = model.predict(test_X)
    predictions = np.round(predictions)

    error = np.abs(test_Y.flatten() - predictions.flatten())
    condition = np.equal(error, 0)
    matches = np.extract(condition, error)

    RMSD = round(math.sqrt(np.sum(error**2)/len(error)),2)
    error_ratio = 100-round((len(matches)/len(error)*100),2)

    return error_ratio, RMSD

In [ ]:
def xp(train_set, test_set, mode):
    print('xp ', mode)
    for i in range(0, N):
        print('    #', str(i))
        train = series_to_supervised(list(train_set[i]), size_of_window, 1)
        test = series_to_supervised(list(test_set[i]), size_of_window, 1) 
        train_X, train_Y, test_X, test_Y = reshape(train, test)
        model = keras_model(train_X.shape[2], train_X.shape[1])
        train_start = datetime.datetime.now()
        model = train_and_save(model, train_X, train_Y, test_X, test_Y, mode, i, save=True)
        train_stop = predict_start = datetime.datetime.now()
        match_ratio, RMSD = predict(model, test_X, test_Y)
        predict_stop = datetime.datetime.now()
        train_duration = train_stop - train_start
        predict_duration = predict_stop - predict_start
        print('RMSD = ', RMSD)
        print(match_ratio, '% error')
        print('training duration = ', train_duration)
        print('prediction duration = ', predict_duration)
        yield match_ratio, RMSD, train_duration, predict_duration

In [ ]:
base_xp = xp(base_train, base_test, 'base')
base_results = list(base_xp)

static_xp = xp(static_train, static_test, 'static')
static_results = list(static_xp)

runtime_xp = xp(runtime_train, runtime_test, 'runtime')
runtime_results = list(runtime_xp)

In [ ]:
def get_errors_and_rmsd(results, errors, rmsd):
    for e,rms,_,_ in results:
        errors.append(e)
        rmsd.append(rms)

        
base_errors = []
base_rmsd = []
get_errors_and_rmsd(base_results, base_errors, base_rmsd)

static_errors = []
static_rmsd = []
get_errors_and_rmsd(static_results, static_errors, static_rmsd)

runtime_errors = []
runtime_rmsd = []
get_errors_and_rmsd(runtime_results, runtime_errors, runtime_rmsd)


plt.figure(4, figsize=(15,6))
plt.plot(base_errors)
mean = np.mean(base_errors)
plt.plot([0, len(base_errors)], [mean, mean], linewidth=5)
plt.plot(static_errors)
mean = np.mean(static_errors)
plt.plot([0, len(static_errors)], [mean, mean], linewidth=5)
plt.plot(runtime_errors)
mean = np.mean(runtime_errors)
plt.plot([0, len(runtime_errors)], [mean, mean], linewidth=5)
plt.show()

plt.figure(5, figsize=(15,6))
plt.plot(base_rmsd)
mean = np.mean(base_rmsd)
plt.plot([0, len(base_rmsd)], [mean, mean], linewidth=5)
plt.plot(static_rmsd)
mean = np.mean(static_rmsd)
plt.plot([0, len(static_rmsd)], [mean, mean], linewidth=5)
plt.plot(runtime_rmsd)
mean = np.mean(runtime_rmsd)
plt.plot([0, len(runtime_rmsd)], [mean, mean], linewidth=5)
plt.show()

In [ ]:
def xp2(train_set, test_set, mode):
    for i in range(0,N):
        model = keras.models.load_model(base_dir + mode + str(i) + '.h5')
        for j in range(0,N):
            print('    using model ', str(i), ' on data coming from ', str(j))
            train = series_to_supervised(list(train_set[j]), size_of_window, 1)
            test = series_to_supervised(list(test_set[j]), size_of_window, 1) 
            train_X, train_Y, test_X, test_Y = reshape(train, test)
            predict_start = datetime.datetime.now()
            match_ratio, RMSD = predict(model, test_X, test_Y)
            predict_stop = datetime.datetime.now()
            predict_duration = predict_stop - predict_start
            print('RMSD = ', RMSD)
            print(match_ratio, '% error')
            print('prediction duration = ', predict_duration)
            yield match_ratio, RMSD, -1, predict_duration #-1 for no training

base_results = xp2(base_train, base_test, 'base')
base_results = list(base_results)

static_results = xp2(static_train, static_test, 'static')
static_results = list(static_results)

runtime_results = xp2(runtime_train, runtime_test, 'runtime')
runtime_results = list(runtime_results)

In [ ]:
def get_errors_and_rmsd_without_self(results, errors, rms, errors_without_self, RMSDs_without_self):
    get_errors_and_rmsd(results, errors, rms)
    for i in range(0,N):
        for j in range(0,N):            
            if(i!=j):
                errors_without_self.append(errors[i*N+j])
                RMSDs_without_self.append(rms[i*N+j])

base_errors = []
base_rmsd = []
base_errors_without_self = []
base_rmsd_without_self = []
get_errors_and_rmsd_without_self(base_results, base_errors, base_rmsd, base_errors_without_self, base_rmsd_without_self)
base_std_error = np.std(base_errors)
base_std_error_without = np.std(base_errors_without_self)
print(base_std_error, ' ', base_std_error_without)

static_errors = []
static_rmsd = []
static_errors_without_self = []
static_rmsd_without_self = []
get_errors_and_rmsd_without_self(static_results, static_errors, static_rmsd, static_errors_without_self, static_rmsd_without_self)                
static_std_error = np.std(static_errors)
static_std_error_without = np.std(static_errors_without_self)
print(static_std_error, ' ', static_std_error_without)

runtime_errors = []
runtime_rmsd = []
runtime_errors_without_self = []
runtime_rmsd_without_self = []
get_errors_and_rmsd_without_self(runtime_results, runtime_errors, runtime_rmsd, runtime_errors_without_self, runtime_rmsd_without_self)                
runtime_std_error = np.std(runtime_errors)
runtime_std_error_without = np.std(runtime_errors_without_self)
print(runtime_std_error, ' ', runtime_std_error_without)

plt.figure(6, figsize=(15,6))
plt.plot(base_errors)
mean = np.mean(base_errors)
plt.plot([0, len(base_errors)], [mean, mean], linewidth=5)
plt.plot(static_errors)
mean = np.mean(static_errors)
plt.plot([0, len(static_errors)], [mean, mean], linewidth=5)
plt.plot(runtime_errors)
mean = np.mean(runtime_errors)
plt.plot([0, len(runtime_errors)], [mean, mean], linewidth=5)
plt.show()

plt.figure(7, figsize=(15,6))
plt.plot(base_errors_without_self)
mean = np.mean(base_errors_without_self)
plt.plot([0, len(base_errors_without_self)], [mean, mean], linewidth=5)
plt.plot(static_errors_without_self)
mean = np.mean(static_errors_without_self)
plt.plot([0, len(static_errors_without_self)], [mean, mean], linewidth=5)
plt.plot(runtime_errors_without_self)
mean = np.mean(runtime_errors_without_self)
plt.plot([0, len(runtime_errors_without_self)], [mean, mean], linewidth=5)
plt.show()

plt.figure(8, figsize=(15,6))
plt.plot(base_rmsd)
mean = np.mean(base_rmsd)
plt.plot([0, len(base_rmsd)], [mean, mean], linewidth=5)
plt.plot(static_rmsd)
mean = np.mean(static_rmsd)
plt.plot([0, len(static_rmsd)], [mean, mean], linewidth=5)
plt.plot(runtime_rmsd)
mean = np.mean(runtime_rmsd)
plt.plot([0, len(runtime_rmsd)], [mean, mean], linewidth=5)
plt.show()

plt.figure(9, figsize=(15,6))
plt.plot(base_rmsd_without_self)
mean = np.mean(base_rmsd_without_self)
plt.plot([0, len(base_rmsd_without_self)], [mean, mean], linewidth=5)
plt.plot(static_rmsd_without_self)
mean = np.mean(static_rmsd_without_self)
plt.plot([0, len(static_rmsd_without_self)], [mean, mean], linewidth=5)
plt.plot(runtime_rmsd_without_self)
mean = np.mean(runtime_rmsd_without_self)
plt.plot([0, len(runtime_rmsd_without_self)], [mean, mean], linewidth=5)
plt.show()

def get_error_rmsd_matrices(errors, rmsd, error_matrix, rmsd_matrix):
    for i in range(0,N):
        for j in range(0,N):
            error_matrix[i][j] = errors[N*i+j]
            rmsd_matrix[i][j] = rmsd[N*i+j]
            
base_error_matrix = np.empty([N,N], dtype=float)
base_rmsd_matrix = np.empty([N,N], dtype=float)
get_error_rmsd_matrices(base_errors, base_rmsd, base_error_matrix, base_rmsd_matrix)

static_error_matrix = np.empty([N,N], dtype=float)
static_rmsd_matrix = np.empty([N,N], dtype=float)
get_error_rmsd_matrices(static_errors, static_rmsd, static_error_matrix, static_rmsd_matrix)

runtime_error_matrix = np.empty([N,N], dtype=float)
runtime_rmsd_matrix = np.empty([N,N], dtype=float)
get_error_rmsd_matrices(runtime_errors, runtime_rmsd, runtime_error_matrix, runtime_rmsd_matrix)

def plot_matrices(error_matrix, rmsd_matrix, mode):       
    #Display error matrix  
    fig, ax = plt.subplots(figsize=(int(N/2), int(N/2)))
    im = ax.imshow(error_matrix, cmap=colormap)
    for i in range(0,N):
        for j in range(0,N):
            text = ax.text(j, i, error_matrix[i, j], ha="center", va="center", color="black", fontsize = 9)

    ax.set_title(mode+" error generalization ", fontsize = 15)
    fig.tight_layout()
    plt.show()
    plt.savefig(mode+'_error_generalization.png', dpi=300)


    #Display RMSD matrix
    fig, ax = plt.subplots(figsize=(int(N/2), int(N/2)))
    im = ax.imshow(rmsd_matrix, cmap=colormap)
    for i in range(0,N):
        for j in range(0,N):
            text = ax.text(j, i, rmsd_matrix[i, j], ha="center", va="center", color="black", fontsize = 9)

    ax.set_title(mode+" RMSD generalization", fontsize = 15)
    fig.tight_layout()
    plt.show()
    plt.savefig(mode+'_rmsd_generalization.png', dpi=300)
    
plot_matrices(base_error_matrix, base_rmsd_matrix, 'base')
plot_matrices(static_error_matrix, static_rmsd_matrix, 'static')    
plot_matrices(runtime_error_matrix, runtime_rmsd_matrix, 'runtime')    

In [ ]:
def xp3(train_set, test_set, mode):
    for i in range(0,N,5):    
        print('training model on protocols', str(i), '-', str(i+4))
        train = series_to_supervised(list(train_set[i]), size_of_window, 1)
        test = series_to_supervised(list(test_set[i]), size_of_window, 1) 
        train_X_block, train_Y_block, test_X, test_Y = reshape(train, test)
        for j in range(i,i+5):
            train = series_to_supervised(list(train_set[j]), size_of_window, 1)
            test = series_to_supervised(list(test_set[j]), size_of_window, 1)
            train_X, train_Y, _, _ = reshape(train, test)
            train_X_block = np.concatenate((train_X_block, train_X), 0)
            train_Y_block = np.concatenate((train_Y_block, train_Y), 0)
        
        model = keras_model(train_X.shape[2], train_X.shape[1])
        train_start = datetime.datetime.now()
        model = train_and_save(model, train_X_block, train_Y_block, test_X, test_Y, mode, i+2000)#FIXME: split train_and_save into 2 def
        train_stop = datetime.datetime.now()
        train_duration = train_stop - train_start
        
        for j in range(0,N):
            print('using model trained on protocols', str(i), '-', str(i+4), ' on data coming from ', str(j))
            train = series_to_supervised(list(train_set[j]), size_of_window, 1)
            test = series_to_supervised(list(test_set[j]), size_of_window, 1) 
            train_X, train_Y, test_X, test_Y = reshape(train, test)
            predict_start = datetime.datetime.now()
            match_ratio, RMSD = predict(model, test_X, test_Y)
            predict_stop = datetime.datetime.now()
            predict_duration = predict_stop - predict_start
            print('RMSD = ', RMSD)
            print(match_ratio, '% error')
            print('training duration = ', train_duration)
            print('prediction duration = ', predict_duration)
            yield match_ratio, RMSD, train_duration, predict_duration

runtime_results = xp3(runtime_train, runtime_test, 'runtime')
runtime_results = list(runtime_results)

static_results = xp3(static_train, static_test, 'static')
static_results = list(static_results)

base_results = xp3(base_train, base_test, 'base')
base_results = list(base_results)

In [ ]:
def get_errors_and_rmsd_without_self2(results, errors, rms, errors_without_self, RMSDs_without_self, self_errors, self_RMSDs, skip=5):
    get_errors_and_rmsd(results, errors, rms)
    s = 0
    r = 0
    for i in range(0,len(errors)):
        if s < skip:
            self_errors.append(errors[i])
            self_RMSDs.append(rms[i])
            s = s + 1            
        else:
            errors_without_self.append(errors[i])
            RMSDs_without_self.append(rms[i])
            r = r + 1           
            if (r >= N):
                r = 0
                s = 0
        
base_errors = []
base_rmsd = []
base_errors_w = []
base_rmsd_w = []
self_base_errors = []
self_base_rmsd = []
get_errors_and_rmsd_without_self2(base_results, base_errors, base_rmsd, base_errors_w, base_rmsd_w, self_base_errors, self_base_rmsd)
base_std_error = np.std(base_errors)
base_std_error_w = np.std(base_errors_w)
print(base_std_error, ' ', base_std_error_w)

static_errors = []
static_rmsd = []
static_errors_w = []
static_rmsd_w = []
self_static_errors = []
self_static_rmsd = []
get_errors_and_rmsd_without_self2(static_results, static_errors, static_rmsd, static_errors_w, static_rmsd_w, self_static_errors, self_static_rmsd)                
static_std_error = np.std(static_errors)
static_std_error_w = np.std(static_errors_w)
print(static_std_error, ' ', static_std_error_w)

runtime_errors = []
runtime_rmsd = []
runtime_errors_w = []
runtime_rmsd_w = []
self_runtime_errors = []
self_runtime_rmsd = []
get_errors_and_rmsd_without_self2(runtime_results, runtime_errors, runtime_rmsd, runtime_errors_w, runtime_rmsd_w, self_runtime_errors, self_runtime_rmsd)                
runtime_std_error = np.std(runtime_errors)
runtime_std_error_w = np.std(runtime_errors_w)
print(runtime_std_error, ' ', runtime_std_error_w)

plt.figure(10, figsize=(15,6))
plt.plot(base_errors)
mean = np.mean(base_errors)
plt.plot([0, len(base_errors)], [mean, mean], linewidth=5)
plt.plot(static_errors)
mean = np.mean(static_errors)
plt.plot([0, len(static_errors)], [mean, mean], linewidth=5)
plt.plot(runtime_errors)
mean = np.mean(runtime_errors)
plt.plot([0, len(runtime_errors)], [mean, mean], linewidth=5)
plt.show()

plt.figure(11, figsize=(15,6))
plt.plot(base_errors_w)
mean = np.mean(base_errors_w)
plt.plot([0, len(base_errors_w)], [mean, mean], linewidth=5)
plt.plot(static_errors_w)
mean = np.mean(static_errors_w)
plt.plot([0, len(static_errors_w)], [mean, mean], linewidth=5)
plt.plot(runtime_errors_w)
mean = np.mean(runtime_errors_w)
plt.plot([0, len(runtime_errors_w)], [mean, mean], linewidth=5)
plt.show()

plt.figure(12, figsize=(15,6))
plt.plot(self_base_errors)
mean = np.mean(self_base_errors)
plt.plot([0, len(self_base_errors)], [mean, mean], linewidth=5)
plt.plot(self_static_errors)
mean = np.mean(self_static_errors)
plt.plot([0, len(self_static_errors)], [mean, mean], linewidth=5)
plt.plot(self_runtime_errors)
mean = np.mean(self_runtime_errors)
plt.plot([0, len(self_runtime_errors)], [mean, mean], linewidth=5)
plt.show()

plt.figure(13, figsize=(15,6))
plt.plot(base_rmsd)
mean = np.mean(base_rmsd)
plt.plot([0, len(base_rmsd)], [mean, mean], linewidth=5)
plt.plot(static_rmsd)
mean = np.mean(static_rmsd)
plt.plot([0, len(static_rmsd)], [mean, mean], linewidth=5)
plt.plot(runtime_rmsd)
mean = np.mean(runtime_rmsd)
plt.plot([0, len(runtime_rmsd)], [mean, mean], linewidth=5)
plt.show()

plt.figure(14, figsize=(15,6))
plt.plot(base_rmsd_w)
mean = np.mean(base_rmsd_w)
plt.plot([0, len(base_rmsd_w)], [mean, mean], linewidth=5)
plt.plot(static_rmsd_w)
mean = np.mean(static_rmsd_w)
plt.plot([0, len(static_rmsd_w)], [mean, mean], linewidth=5)
plt.plot(runtime_rmsd_w)
mean = np.mean(runtime_rmsd_w)
plt.plot([0, len(runtime_rmsd_w)], [mean, mean], linewidth=5)
plt.show()

plt.figure(15, figsize=(15,6))
plt.plot(self_base_rmsd)
mean = np.mean(self_base_rmsd)
plt.plot([0, len(self_base_rmsd)], [mean, mean], linewidth=5)
plt.plot(self_static_rmsd)
mean = np.mean(self_static_rmsd)
plt.plot([0, len(self_static_rmsd)], [mean, mean], linewidth=5)
plt.plot(self_runtime_rmsd)
mean = np.mean(self_runtime_rmsd)
plt.plot([0, len(self_runtime_rmsd)], [mean, mean], linewidth=5)
plt.show()

def get_error_rmsd_matrices2(errors, rmsd, error_matrix, rmsd_matrix, length=N):
    for i in range(0, len(errors)):
        row = math.floor(i/length)
        col = i % length
        error_matrix[row][col] = errors[i]
        rmsd_matrix[row][col] = rmsd[i]
            
x = math.floor(len(base_errors)/N)
y = N
base_error_matrix = np.empty([x,y], dtype=float)
base_rmsd_matrix = np.empty([x,y], dtype=float)
get_error_rmsd_matrices2(base_errors, base_rmsd, base_error_matrix, base_rmsd_matrix)

x = math.floor(len(static_errors)/N)
static_error_matrix = np.empty([x,y], dtype=float)
static_rmsd_matrix = np.empty([x,y], dtype=float)
get_error_rmsd_matrices2(static_errors, static_rmsd, static_error_matrix, static_rmsd_matrix)

x = math.floor(len(runtime_errors)/N)
runtime_error_matrix = np.empty([x,y], dtype=float)
runtime_rmsd_matrix = np.empty([x,y], dtype=float)
get_error_rmsd_matrices2(runtime_errors, runtime_rmsd, runtime_error_matrix, runtime_rmsd_matrix)


def plot_matrices2(error_matrix, rmsd_matrix, mode, x=N, y=N):       
    #Display error matrix  
    fig, ax = plt.subplots(figsize=(int(N/2), int(N/2)))
    im = ax.imshow(error_matrix, cmap=colormap)
    for i in range(0,x):
        for j in range(0,y):
            text = ax.text(j, i, error_matrix[i, j], ha="center", va="center", color="black", fontsize = 9)

    ax.set_title(mode+" error generalization ", fontsize = 15)
    fig.tight_layout()
    plt.show()
    plt.savefig(mode+'_error_generalization.png', dpi=300)


    #Display RMSD matrix
    fig, ax = plt.subplots(figsize=(int(N/2), int(N/2)))
    im = ax.imshow(rmsd_matrix, cmap=colormap)
    for i in range(0,x):
        for j in range(0,y):
            text = ax.text(j, i, rmsd_matrix[i, j], ha="center", va="center", color="black", fontsize = 9)

    ax.set_title(mode+" RMSD generalization", fontsize = 15)
    fig.tight_layout()
    plt.show()
    plt.savefig(mode+'_rmsd_generalization.png', dpi=300)

plot_matrices2(base_error_matrix, base_rmsd_matrix, 'base', x=5)
plot_matrices2(static_error_matrix, static_rmsd_matrix, 'static', x=5)    
plot_matrices2(runtime_error_matrix, runtime_rmsd_matrix, 'runtime', x=5)    